In [13]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기
#Step 1. 필요한 모듈을 로딩합니다                  
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller as ca
import pandas as pd 
import time

#Step 2. 사용자에게 검색 관련 정보들을 입력 받고 3. 파일 이름 정하기
query_txt ='빅데이터'
ft_name = "c:\\py_temp\\riss.txt"
fc_name = "c:\\py_temp\\riss.csv"
fx_name = "c:\\py_temp\\riss.xls"
collect_cnt = int(input())
next_num = collect_cnt//100 #다음으로 갈지 말지~
#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.riss.kr/')
time.sleep(2)
driver.maximize_window()
main = driver.window_handles
for i in main:
    if i !=main[0]:
        driver.switch_to.window(i)
        driver.close()
driver.switch_to.window(driver.window_handles[0]) # driver.switch_to.window(main[0])도 가능
time.sleep(1)
#Step 5. 자동으로 검색어 입력 후 6 학위논문 클릭
driver.find_element(By.ID,'query').send_keys(query_txt+ '\n')
driver.find_element(By.LINK_TEXT,'학위논문').click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
soup_1 = BeautifulSoup(driver.page_source, 'html.parser')
content_1 = soup_1.find('div','srchResultListW').find_all('li')


#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('키워드 %s (으)로  %s 건 검색' %(query_txt,total_cnt))
collect_page_cnt = math.ceil(collect_cnt / 10)


#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1
def fun(next_num):
    global no, collect_page_cnt
    for _ in range(next_num):
        for a in range(1, collect_page_cnt + 1) :
            time.sleep(1)        # 페이지 변경 전 1초 대기 
            soup_2 = BeautifulSoup(driver.page_source, 'html.parser')
            content_2 = soup_2.find('div','srchResultListW').find_all('li')

            for b in content_2 : 
                #1. 논문제목 있을 경우만
                try :
                    title = b.find('p','title').get_text()
                except :
                    continue
                else :
                    f = open(ft_name, 'a' , encoding="UTF-8")

                    print('1.번호:',no)
                    no2.append(no)
                    f.write('\n'+'1.번호:' + str(no))

                    print('2.논문제목:',title)
                    title2.append(title)
                    f.write('\n' + '2.논문제목:' + title)

                    writer = b.find('span','writer').get_text()
                    print('3.저자:',writer)
                    writer2.append(writer)
                    f.write('\n' + '3.저자:' + writer)

                    org = b.find('span','assigned').get_text()
                    print('4.소속기관:' , org)
                    org2.append(org)
                    f.write('\n' + '4.소속기관:' + org + '\n')

                    f.close( )

                    no += 1
                    print("\n")

                    if no > collect_cnt :
                        return
            

            b = str(a+1)

            try :
                driver.find_element(By.LINK_TEXT ,'%s' %b).click() 
            except :
                driver.find_element(By.CLASS_NAME, 'next1').click()
            time.sleep(1)        # 페이지 변경 전 1초 대기 
fun(next_num)
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8" , engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

 112


키워드 빅데이터 (으)로  5,870 건 검색
1.번호: 1
2.논문제목: 동형암호를 이용한 개인정보 등 빅데이터의 범죄분석 활용방안 연구
3.저자: 정대희
4.소속기관: 서울대학교 대학원


1.번호: 2
2.논문제목: 빅데이터를 활용한 대학구조개혁 평가의 키워드 및 토픽 분석
3.저자: 김지은
4.소속기관: 서울대학교 대학원


1.번호: 3
2.논문제목: 조직, 개인 그리고 경영품질 관점에서 기업의 빅데이터 활용의도에 영향을 미치는 핵심요인에 관한 연구
3.저자: 신수행
4.소속기관: 전남대학교


1.번호: 4
2.논문제목: 빅데이터 분석의 사용이 은행 CRM 가치 창출에 미치는 영향 : 고객 상호관계 관점으로
3.저자: 박성수
4.소속기관: 성균관대학교 경영전문대학원


1.번호: 5
2.논문제목: 빅데이터 분석 사업의 정보시스템 감리에 대한 연구
3.저자: 박성남
4.소속기관: 건국대학교 정보통신대학원


1.번호: 6
2.논문제목: 빅데이터 활용 탐구 수업의 설계 효과 : 빅데이터 활용 역량과 학습만족도를 중심으로
3.저자: 윤한나
4.소속기관: 한국교원대학교 교육대학원


1.번호: 7
2.논문제목: IPA기법을 활용한 제조분야의 빅데이터 수용도 분석 : IT 분야 제조 A社 중심
3.저자: 주혁
4.소속기관: 성균관대학교 일반대학원


1.번호: 8
2.논문제목: 기후변화 정책 의사결정 지원을 위한 빅데이터 분석
3.저자: 정예림
4.소속기관: 부산대학교


1.번호: 9
2.논문제목: 사회과학의 텍스트로서 빅데이터
3.저자: 정준석
4.소속기관: 경기대학교 정치전문대학원


1.번호: 10
2.논문제목: 빅데이터를 이용한 부산항신항 ITT 효율적인 운영방안에 관한 연구
3.저자: 백용주
4.소속기관: 한국해양대학교 해양금융물류대학원


1.번호: 11
2.논문제목: 음식관광 빅데이터를 이용한 관광목적지 브랜드 개성에 관한 연구: 방한 중국인 관광객을 중심으로
3.저자: 연진
4.소속기관: 세종대학교 대학원


1.번호: 12
2.논문제목: 빅

/Users/mujae/Desktop/anaconda3/lib/python3.10/site-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [17]:
type(df['번호'])

pandas.core.series.Series